In [15]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv("../data/clustered_df.csv")

In [20]:
data.head()

,X,Y,day,Month,Hour,n_days,PdDistrict,Clusters
622312,-0.871797,-0.055218,-0.066074,1.331962,0.241546,-0.918547,5,0
63599,-0.266478,1.280572,-0.179966,-0.126598,0.394109,1.447257,4,0
32716,1.905681,-1.337436,0.958949,1.331962,-0.368703,1.570342,0,0
81367,-1.939273,0.011803,-0.749423,-1.001734,0.546671,1.374010,5,0
391924,1.152978,0.220264,-0.863315,-0.126598,-2.046887,0.063870,7,1


In [21]:
X_new = data.drop(columns=['Clusters'])
y = data['Clusters']

In [22]:
from sklearn.model_selection import train_test_split

# Split into Train + Temp (60% train_val, 40% test)
x_train, x_temp, y_train, y_temp = train_test_split(X_new, y, test_size=0.4, random_state=43)

# Split the temp into test and validation (20% test) and  (20% Validation)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=43)

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Train the model on training data
tree_model = DecisionTreeClassifier(max_depth=15, min_samples_split=100, min_samples_leaf=50, criterion='entropy')
tree_model.fit(x_train, y_train)

# 2. Evaluate on Train data
y_pred_tree_train = tree_model.predict(x_train)
print("Decision Tree (Train) Results:")
print("Accuracy of Decision Tree (Train):", accuracy_score(y_train, y_pred_tree_train))
print(classification_report(y_train, y_pred_tree_train, zero_division=0))
print("****************************************")

# 3. Evaluate on Validation data
y_pred_tree_val = tree_model.predict(x_val)
print("Decision Tree (Validation) Results:")
print("Accuracy of Decision Tree (Validation):", accuracy_score(y_val, y_pred_tree_val))
print(classification_report(y_val, y_pred_tree_val, zero_division=0))
print("****************************************")

# 4. Evaluate on Test data
y_pred_tree_test = tree_model.predict(x_test)
print("Decision Tree (Test) Results:")
print("Accuracy of Decision Tree (Test):", accuracy_score(y_test, y_pred_tree_test))
print(classification_report(y_test, y_pred_tree_test, zero_division=0))

Decision Tree (Train) Results:
Accuracy of Decision Tree (Train): 0.9556
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9175
           1       0.87      0.90      0.89      2306
           2       0.93      0.90      0.91      3519

    accuracy                           0.96     15000
   macro avg       0.93      0.93      0.93     15000
weighted avg       0.96      0.96      0.96     15000

****************************************
Decision Tree (Validation) Results:
Accuracy of Decision Tree (Validation): 0.946
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1224
           1       0.88      0.86      0.87       326
           2       0.87      0.91      0.89       450

    accuracy                           0.95      2000
   macro avg       0.92      0.92      0.92      2000
weighted avg       0.95      0.95      0.95      2000

****************************************
D

In [ ]:
from sklearn.tree import DecisionTreeClassifier

rf_best = RandomForestClassifier(
    bootstrap=True,
    max_depth=70,
    max_features='sqrt',
    min_samples_leaf=4,
    min_samples_split=10,
    n_estimators=400,
    random_state=42
)

# Train the model on train data
rf_best.fit(x_train, y_train)
# Predict on train data
y_pred_rf_train = rf_best.predict(x_train)
print("RandomForest (Train) Results:")
print("Accuracy of RandomForest (Train):", accuracy_score(y_train, y_pred_rf_train))
print("Classification Report (Train):\n", classification_report(y_train, y_pred_rf_train, zero_division=0))
print("****************************************")

# Evaluate on Validation data
y_pred_rf_val = rf_best.predict(x_val)
print("RandomForest (Validation) Results:")
print("Accuracy of RandomForest (Validation):", accuracy_score(y_val, y_pred_rf_val))
print("Classification Report (Validation):\n", classification_report(y_val, y_pred_rf_val, zero_division=0))
print("****************************************")

# Evaluate on Test data
y_pred_rf_test = rf_best.predict(x_test)
print("RandomForest (Test) Results:")
print("Accuracy of RandomForest (Test):", accuracy_score(y_test, y_pred_rf_test))
print("Classification Report (Test):\n", classification_report(y_test, y_pred_rf_test, zero_division=0))

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf_test)
labels = [f"Cluster {i}" for i in range(cm.shape[0])]
plt.figure(figsize=(10, 7))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=labels,
    yticklabels=labels,
    linewidths=0.5,
    linecolor='gray'
)
plt.title("Confusion Matrix - Random Forest (Test Data)", fontsize=14, weight='bold')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()